In [1]:
from PIL import Image
from rl_benchmarks.models import iBOTViT
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
import pathlib
from tqdm import tqdm
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
from multiprocessing import Pool
import umap
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from rl_benchmarks.utils.linear_evaluation import get_binary_class_metrics, get_bootstrapped_metrics

from PIL import Image
import pathlib
from tqdm import tqdm
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
from multiprocessing import Pool
# import umap
import numpy as np
import matplotlib.pyplot as plt
import sklearn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

from sklearn.metrics import f1_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, cohen_kappa_score, accuracy_score
import torch.nn.functional as F
import sys
import time

import shutil
import os

# import albumentations as A
# from albumentations.pytorch import ToTensorV2
import cv2
import time
from sklearn.preprocessing import label_binarize

import timm
from metrics import report
from pathlib import Path

import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset, Dataset
from tqdm import tqdm
from rl_benchmarks.metrics import *

from rl_benchmarks.trainers.torch_trainer import TorchTrainer
from rl_benchmarks.models.slide_models.meanpool import MeanPool
from rl_benchmarks.models.slide_models.chowder import Chowder
from rl_benchmarks.models.slide_models.dsmil import DSMIL
from rl_benchmarks.models.slide_models.abmil import ABMIL
from rl_benchmarks.models.slide_models.hiptmil import HIPTMIL
from rl_benchmarks.models.slide_models.transmil import TransMIL

from pathlib import Path
from metrics import report

import sys
sys.path.append('/home/yuxin/bme/BCaCAD/model')
from patch_based_test.img import QiLuROI

def get_model(model_name, in_dim, out_dim):
    if model_name == 'abmil':
        model = ABMIL(in_dim, out_dim, d_model_attention=128,temperature= 1.0, mlp_hidden= [128, 64])
    elif model_name == 'chowder':
        model = Chowder(in_dim, out_dim, n_top= 2,n_bottom= 2,tiles_mlp_hidden= [128], mlp_hidden= [128, 64])
    elif model_name == 'dsmil':
        model = DSMIL(in_dim, out_dim, d_tiles_values= 32,d_tiles_queries= 32,passing_values= False,tiles_scores_mlp_hidden=[200,100],
                        tiles_queries_mlp_hidden=[200,100], mlp_hidden=[200,100])
    elif model_name == 'hiptmil':
        model = HIPTMIL(in_dim, out_dim)
    elif model_name == 'transmil':
        model = TransMIL(in_dim, out_features=out_dim)
    elif model_name == 'meanpool':
        model = MeanPool(in_dim, out_dim)
    else:
        raise 'model not found'
    return model


In [2]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
num_classes = [3,3]
img_size = patch_size = 384
data_trans = {
    "train": transforms.Compose([
                                transforms.Resize(img_size),
                                transforms.ColorJitter(),
                                transforms.RandomHorizontalFlip(),
                                # transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    "test": transforms.Compose([
                                transforms.Resize((img_size,img_size)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

In [3]:
class Feature_Fly(Dataset):
    def get_label(self, type):
        D = {
            'normal':[0,0],
            'dcis-1':[1,0],
            'dcis-2':[1,1],
            'dcis-3':[1,2],
            'ibc-1':[2,0],
            'ibc-2':[2,1],
            'ibc-3':[2,2],
        }
        label =  D[type]
        return label[0],label[1]
    
    def __init__(self, image_dir, phase='test', train_tile=None):
        folder = Path(image_dir)
        # self.im_paths = list(folder.rglob('**/*.*'))
        # self.labels = [self.get_label(path.parent.name) for path in self.im_paths]
        self.wsi_dirs = list(folder.glob('*/*'))
        self.phase=phase
        self.train_tile = train_tile
        self.size = 336
        self.bs = 16
        self.src_mag = 10
        self.tar_mag = 10
    
        
    def __len__(self):
        return len(self.wsi_dirs)
    def __getitem__(self, item):
        wsi_dir = self.wsi_dirs[item]
        im_paths = list(wsi_dir.rglob('**/*.png'))
        wsi_patches = []
        for im_path in im_paths:
            im = QiLuROI(str(im_path), self.src_mag, self.tar_mag, self.size)
            im.setIterator(self.size)
            patches = [p for p in im]
            wsi_patches.extend(patches)
        patches = wsi_patches
        if self.phase == 'train':
            indices = np.random.choice(len(patches), self.train_tile, replace=True)
            patches = [patches[i] for i in indices]
        patches = [data_trans[self.phase](p) for p in patches]
        bs = self.bs
        for i in range(0, len(patches), bs):
            x = torch.stack(patches[i:i+bs], dim=0)
            x = x.cuda(1)
            y = feature_model.module.base_model(x)
            if i == 0:
                features = y#.detach().cpu().numpy()
            else:
                features = torch.concatenate([features, y], dim=0)
                #np.concatenate([features, y.detach().cpu().numpy()], axis=0)
        label = self.get_label(wsi_dir.parent.name)
        return features.detach().cpu(), label
    
    @staticmethod
    def collate_fn(batch):
        # 官方实现的default_collate可以参考
        # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
        images,  labels = tuple(zip(*batch))

        images = torch.stack(images, dim=0)
        labels = torch.as_tensor(labels)
        # masks = torch.as_tensor(masks)

        return images, labels

class Feature(Dataset):
    def get_label(self, type):
        D = {
            'normal':[0,0],
            'dcis-1':[1,0],
            'dcis-2':[1,1],
            'dcis-3':[1,2],
            'ibc-1':[2,0],
            'ibc-2':[2,1],
            'ibc-3':[2,2],
        }
        label =  D[type]
        return label[0],label[1]
    
    def __init__(self, feature_dir, phase='test'):
        folder = Path(feature_dir)
        feature_paths = list(folder.rglob('**/*.npy'))
        self.labels = [self.get_label(path.parent.name) for path in feature_paths]
        self.features = [np.load(path) for path in feature_paths]
        self.phase=phase
        
    def __len__(self):
        return len(self.features)
    def __getitem__(self, item):
        features = self.features[item]
        if self.phase == 'train':
            indices = np.random.choice(features.shape[0], 8, replace=True)
            features = np.stack([features[i] for i in indices], axis=0)
        return features, self.labels[item]
    
    @staticmethod
    def collate_fn(batch):
        # 官方实现的default_collate可以参考
        # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
        images,  labels = tuple(zip(*batch))

        images = torch.stack(images, dim=0)
        labels = torch.as_tensor(labels)
        # masks = torch.as_tensor(masks)

        return images, labels


In [4]:
cfg = dict(
    epoch=2,
    bs=8,
    train_tiles = 32,
)
criterion = nn.CrossEntropyLoss()
merics = {'acc': compute_multiclass_accuracy, 'auc': compute_mean_one_vs_all_auc}
input_dim = 768
out_dim = [3,3]
tasks = ['type', 'nonibc', 'ibc']
class_names = {
    'type': ['Normal', 'nonIBC', 'IBC'],
    'nonibc': ['Low', 'Medium', 'High'],
    'ibc': ['Low', 'Medium', 'High'],
}

In [5]:
device = "cuda"

In [6]:
class WSI(Dataset):
    def get_label(self, type):
        D = {
            'normal':[0,0],
            'dcis-1':[1,0],
            'dcis-2':[1,1],
            'dcis-3':[1,2],
            'ibc-1':[2,0],
            'ibc-2':[2,1],
            'ibc-3':[2,2],
        }
        label =  D[type]
        return label[0],label[1]
    
    def __init__(self, feature_dir, phase='test'):
        folder = Path(feature_dir)
        wsi_dirs = list(folder.glob('*'))
        # self.labels = [self.get_label(path.parent.name) for path in wsi_dirs]
        self.labels = []
        self.names = []
        for path in wsi_dirs:
            # label = path.parent.name
            # cvtlabel = self.get_label(label)
            self.names.append(path.name)
            self.labels.append([0,0])
        # self.features = [np.load(path) for path in feature_paths]
        self.features = []
        for wsi_dir in wsi_dirs:
            feature_paths = list(wsi_dir.rglob('**/*.npy'))
            features = []
            for path in feature_paths:
                patch_features = torch.from_numpy(np.load(path)).to(device).unsqueeze(0)
                # roi_feats = feature_model.extract_features(patch_features)
                features.extend(patch_features)
            self.features.append(features)
        self.wsis = wsi_dirs
        self.phase=phase
        
    def __len__(self):
        return len(self.wsis)
    def __getitem__(self, item):
        features = self.features[item]
        # wsi = self.wsis[item]
        if self.phase == 'train':
            indices = np.random.choice(len(features), 6, replace=True)
            features = torch.cat([features[i] for i in indices], dim=0)
            return features, self.labels[item]
        features = torch.cat(features, dim=0)
        return features, self.labels[item]
    
    @staticmethod
    def collate_fn(batch):
        # 官方实现的default_collate可以参考
        # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
        images,  labels = tuple(zip(*batch))

        images = torch.stack(images, dim=0)
        labels = torch.as_tensor(labels)
        # masks = torch.as_tensor(masks)

        return images, labels

In [7]:
model_names = [ 'abmil']
models={}
device = 'cuda'
model_dir = Path('/mnt/hd0/project/bcacad/model/wsi_models/model2')
for model_name in model_names:
    model = get_model(model_name, input_dim, out_dim).to(device)
    model = nn.DataParallel(model, device_ids=[0,1], output_device=0)
    model.load_state_dict(torch.load(model_dir / f'{model_name}.pth'))
    models[model_name] = model.to(device)

In [8]:
# testing
pha='test'
device = 'cuda'
# test_cohorts = ['suqh_all_patch', 'qduh', 'shsu', 'bracs', 'bcnb', 'bach', 'apght']
# test_cohorts = [ 'suqh_all_patch']
# model_names = [ 'abmil', 'meanpool','chowder', 'transmil']
model_names = ['abmil']
# load_dir = Path('/mnt/hd0/project/bcacad/model/roi_models/model2')
feature_root = Path('/mnt/hd0/project/bcacad/model/100_wsi')
# for cohort in test_cohorts:
# for model_name in model_names:
    # model = get_model(model_name, input_dim, out_dim).to(device)
model = models[model_name]
# ckpt = torch.load(load_dir / f'{model_name}.pth')
# model.load_state_dict(ckpt)
# model = model.to(device)

test_set = WSI(feature_root, 'test')
trainer = TorchTrainer(model, criterion, merics, device=device, num_epochs=50)
res = trainer.predict(test_set)

# labels = np.array(res[0])
wsi_names = test_set.names
probs = np.array(res[1])
preds = np.array(res[2])

In [13]:
# use softmax to probs
import numpy as np

def softmax(probs):
    # Step 1: Exponentiate each element
    exp_probs = np.exp(probs)
    
    # Step 2: Sum the exponentials for each row
    sum_exp_probs = np.sum(exp_probs, axis=1, keepdims=True)
    
    # Step 3: Divide by the sum
    softmax_probs = exp_probs / sum_exp_probs
    
    return softmax_probs
probs[0] = softmax(probs[0])
probs[1] = softmax(probs[1])

In [10]:
# save_root = Path('/mnt/hd0/project/bcacad/model/wsi_models/model2')
# if not save_root.exists():
#     save_root.mkdir()
# for model_name in model_names:
#     torch.save(models[model_name].state_dict(), save_root / f'{model_name}.pth')
#     print(f'save {model_name}.pth')

In [14]:
import pandas as pd
type_data = {
    'name': wsi_names,
    'pred' : preds[0],
    'prob-0' : probs[0][:,0],
    'prob-1' : probs[0][:,1],
    'prob-2' : probs[0][:,2],
}
df = pd.DataFrame(type_data)
df.to_excel('100_wsi_type_preds_v3.xlsx', index=False)

In [15]:
grade_data = {
    'name': wsi_names,
    'pred' : preds[1],
    'prob-0' : probs[1][:,0],
    'prob-1' : probs[1][:,1],
    'prob-2' : probs[1][:,2],
}
df = pd.DataFrame(grade_data)
df.to_excel('100_wsi_grade_preds_v3.xlsx', index=False)